In [1]:
from openpyxl import load_workbook

# Загружаем файл и нужный лист
wb = load_workbook("Все_ТС_Поездки_27.03.2025_16-52-22.xlsx")
ws = wb["Поездки"]

# Пройдемся по всем строкам, начиная с 2 (если есть заголовок)
for row in ws.iter_rows(min_row=2):
    # Столбец D (index 3) и F (index 5) — т.е. D = 4-я ячейка, F = 6-я
    for col_idx in [3, 5]:  # D и F
        cell = row[col_idx]
        if cell.hyperlink:
            cell.value = cell.hyperlink.target  # Заменяем текст на ссылку

# Сохраняем результат в новый файл
wb.save("Поездки_с_гиперссылками.xlsx")

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
import pandas as pd

df = pd.read_excel('Поездки_с_гиперссылками.xlsx', sheet_name='Поездки')

pattern = r'q=(-?\d+(?:\.\d+)?),(-?\d+(?:\.\d+)?)'

df[['latitude_нач', 'longitude_нач']] = df['Нач. положение'].str.extract(pattern)

df[['latitude_нач', 'longitude_нач']] = df[['latitude_нач', 'longitude_нач']].astype(float)

df[['latitude_конеч', 'longitude_конеч']] = df['Конеч. положение'].str.extract(pattern)

df[['latitude_конеч', 'longitude_конеч']] = df[['latitude_конеч', 'longitude_конеч']].astype(float)

df = df[['№', 'Группировка', 'Начало', 'latitude_нач', 'longitude_нач', 'Конец', 'latitude_конеч', 'longitude_конеч']]

In [12]:
df

,№,Группировка,Начало,latitude_нач,longitude_нач,Конец,latitude_конеч,longitude_конеч
0,1,313 камри Сагадиев,00:53:00,51.08689,71.41246,16:21:57,51.11097,71.39496
1,1.1,2025-03-17,00:53:00,51.08689,71.41246,20:45:24,51.11110,71.39500
2,1.1.1,313 камри Сагадиев,00:53:00,51.08689,71.41246,01:02:12,51.11112,71.39517
3,1.1.2,313 камри Сагадиев,07:30:44,51.11112,71.39517,07:54:19,51.11172,71.39511
4,1.1.3,313 камри Сагадиев,09:29:41,51.11132,71.39506,09:35:40,51.10295,71.40411
...,...,...,...,...,...,...,...,...
8805,102.11.9,Джексембаев 353,13:03:38,48.88398,83.32853,13:22:45,49.02346,83.36527
8806,102.11.10,Джексембаев 353,13:56:17,49.02346,83.36527,14:02:24,49.00574,83.36596
8807,102.11.11,Джексембаев 353,14:28:08,49.00574,83.36596,15:54:23,49.57962,82.71806
8808,102.11.12,Джексембаев 353,16:00:55,49.57962,82.71806,16:51:36,49.94275,82.62243


In [13]:
df.rename(columns={'№': 'num'}, inplace=True)

df[['lvl1', 'lvl2', 'lvl3']] = df['num'].str.split('.', n=2, expand=True)

In [14]:
mask = df['lvl2'].notna()

# Применяем transform ТОЛЬКО к строкам, попадающим в mask
df.loc[mask, 'Группировка'] = (
    df[mask]
    .groupby(['lvl1', 'lvl2'])['Группировка']
    .transform('first')
)

In [15]:
# 1. Преобразуем «Начало» и «Конец» в формат времени (по умолчанию дата будет 1970-01-01)
df['Начало'] = pd.to_datetime(df['Начало'], format='%H:%M:%S', errors='coerce')
df['Конец']  = pd.to_datetime(df['Конец'],  format='%H:%M:%S', errors='coerce')

# 2. Маска: берём только те строки, у которых lvl2 не NaN/None
mask = df['lvl2'].notna()

# 3. Из «Группировка» извлекаем дату (год-месяц-день),
#    а из «Начало» и «Конец» — время (часы:минуты:секунды).
#    В итоге мы склеим в одну дату-время.

date_part = pd.to_datetime(df.loc[mask, 'Группировка'], errors='coerce').dt.date
start_times = df.loc[mask, 'Начало'].dt.time
end_times   = df.loc[mask, 'Конец'].dt.time

# 4. Склеиваем дату + время в полноценный datetime64[ns].
#    Превращаем date_part и start_times/end_times обратно в строку, чтобы сложить их через пробел.
df.loc[mask, 'Начало'] = pd.to_datetime(
    date_part.astype(str) + ' ' + start_times.astype(str),
    errors='coerce'
)
df.loc[mask, 'Конец'] = pd.to_datetime(
    date_part.astype(str) + ' ' + end_times.astype(str),
    errors='coerce'
)

In [16]:
# Пусть у вас есть столбцы: 'lvl1', 'Группировка' и т.д.

df['Группировка'] = (
    df.groupby('lvl1')['Группировка']
      .transform('first')  # берём значение из первой строки группы
)


In [17]:
df.drop('num', axis = 1, inplace=True)
df = df.dropna().reset_index(drop=True)

In [18]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

def cluster_coordinates_dbscan(df, eps=0.001):
    """
    Для каждой группы (столбец 'Группировка') с помощью DBSCAN
    группируем все координаты: и 'latitude_нач, longitude_нач', и 'latitude_конеч, longitude_конеч'.
    eps=0.0002 ~ 22 метров (т.к. 1 градус ~ 111 км).
    """

    # Делаем копию, чтобы не портить исходный DataFrame
    df_clustered = df.copy()

    # Сортируем (не обязательно, но если есть логика по времени)
    df_clustered.sort_values(by=["Группировка", "Начало"], inplace=True)

    # Разбиваем на группы
    grouped = df_clustered.groupby("Группировка", group_keys=True)

    # Сохраним результаты в список, потом склеим
    dfs = []
    for group_name, group_data in grouped:
        # Имеем один агент (группу). Нужно кластеризовать все точки:
        # как "начальные", так и "конечные".
        # Сформируем общий список координат (N*2 строки), чтобы
        # DBSCAN «видел» все точки сразу.
        
        # Собираем все "начальные" и "конечные" координаты в один массив
        # + запоминаем индексы и метку (начальная или конечная),
        # чтобы потом вернуть значения на место.
        coords = []
        index_and_type = []  # (индекс, 'start') или (индекс, 'end')

        for idx, row in group_data.iterrows():
            coords.append([row["latitude_нач"], row["longitude_нач"]])
            index_and_type.append((idx, 'start'))

            coords.append([row["latitude_конеч"], row["longitude_конеч"]])
            index_and_type.append((idx, 'end'))

        coords = np.array(coords)
        
        # Прогоняем через DBSCAN
        # eps=0.0002 -> порог, min_samples=1, чтобы из 1 точки тоже формировался кластер
        clusterer = DBSCAN(eps=eps, min_samples=1)
        labels = clusterer.fit_predict(coords)

        # Теперь у нас есть массив labels (по одной метке на каждую точку)
        # Точки с одинаковой меткой принадлежат одному кластеру.
        # Нужно заменить координаты внутри каждого кластера на "эталон" 
        # (например, на первую точку кластера или на среднее).
        
        # Определим центры кластеров как среднее по кластерам (можно брать первую точку)
        unique_labels = np.unique(labels)
        cluster_centers = {}

        for lab in unique_labels:
            cluster_points = coords[labels == lab]
            # Средняя координата внутри кластера:
            center_lat = cluster_points[:, 0].mean()
            center_lon = cluster_points[:, 1].mean()
            cluster_centers[lab] = (center_lat, center_lon)
        
        # Записываем "очищенные" координаты обратно
        for i, lab in enumerate(labels):
            (idx, coord_type) = index_and_type[i]
            new_lat, new_lon = cluster_centers[lab]

            if coord_type == 'start':
                group_data.at[idx, "latitude_нач"]   = new_lat
                group_data.at[idx, "longitude_нач"]  = new_lon
            else:  # 'end'
                group_data.at[idx, "latitude_конеч"] = new_lat
                group_data.at[idx, "longitude_конеч"] = new_lon

        dfs.append(group_data)

    # Склеиваем все обработанные группы
    df_clustered = pd.concat(dfs).sort_index()
    return df_clustered

# Пример использования:
df = cluster_coordinates_dbscan(df, eps=0.0002)

In [19]:
df.to_csv('data.csv', index=False)